<a href="https://colab.research.google.com/github/romapavelko01/NLP_SDLC_project/blob/classifications/SGDClassifier_for_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
filename = "/content/drive/MyDrive/SDLC/news_analysis_project/data/final_news_category_dataset.json"
df = pd.read_json(filename, orient='split')
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


# Text preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re
import string


def cleaning_function(sentence):
    """
    Function needed to perform data preprocessing: removing punctuation symbols,
    stop_words and other random things in order to obtain clean text
    """
    # the following line removes numbers from text
    result = re.sub(r'\d+', '', sentence.lower())

    # the following line removes any punctuation from the text
    result = result.translate(str.maketrans('','',string.punctuation))
    return [word for word in result.split() if not word in stop_words]


def collect_words(lst):
    """
    Given a list of lists of words,
    return a list of all the words in a list representation of the text.
    """
    return [word for sentence in lst for word in sentence]

In [ ]:
df['processed_description'] = df['short_description'].apply(lambda x: ' '.join(cleaning_function(x)))
df['processed_headline'] = df['headline'].apply(lambda x: ' '.join(cleaning_function(x)))
df['full_text'] = df['headline'] + df['short_description']
df['processed_full_text'] = df['processed_headline'] + df['processed_description']

# Splitting the data into train/test set with identical distribution of labels for processed and raw datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'category'], df['category'], 
                                                    test_size=0.2, stratify=df.category,
                                                    random_state=1)

processed_split_description = (X_train['processed_description'], X_test['processed_description'], y_train, y_test)
processed_split_headline = (X_train['processed_headline'], X_test['processed_headline'], y_train, y_test)
processed_split_full_text = (X_train['processed_full_text'], X_test['processed_full_text'], y_train, y_test)

raw_split_description = (X_train['short_description'], X_test['short_description'], y_train, y_test)
raw_split_headline = (X_train['headline'], X_test['headline'], y_train, y_test)
raw_split_full_text = (X_train['full_text'], X_test['full_text'], y_train, y_test)

# Function for recording accuracy results for each combination of vectorizer parameters into a dataframe 

In [ ]:
results_df = pd.DataFrame(columns=['Classifier', 'By', 'Preprocessed', 'Vectorizer', 'Ngram', 'TopKFeatures', 'TrainAccuracy', 'TestAccuracy'])

def write_df(dataset, preprocessed, by, clf, vect, ngram=(1, 1), topk=6000, display=True):
    """
    Function to calculate accuracies for the given dataset split, 
    whether it is raw or preprocessed; 
    by - in ['headline', 'descirption', 'full_text']
    classifier, 
    vectorizer, ngram parameter, and how many top features to consider;
    record calculated accuracies to the results_df;
    and, once display is set to True, print classification accuracies
    """
    global results_df

    X_train, X_test, y_train, y_test = dataset

    vectorizer = vect(ngram_range=ngram)
    x_train_ = vectorizer.fit_transform(X_train)

    # Vectorize validation texts.
    x_val = vectorizer.transform(X_test)
    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(topk, x_train_.shape[1]))
    selector.fit(x_train_, y_train)
    x_train = selector.transform(x_train_).astype('float32')
    x_val = selector.transform(x_val).astype('float32')   

    clf = clf()
    clf.fit(x_train, y_train)
    y_pred_test = clf.predict(x_val)
    y_pred_train = clf.predict(x_train)
    train_acc, test_acc = accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test)
    if display:
        print(f"Train classification accuracy = {train_acc},\n Test classification accuracy = {test_acc}")

    results_df = results_df.append(pd.DataFrame({
        'Classifier': [clf.__class__.__name__],
        'By': [by],
        'Preprocessed': [preprocessed],
        'Vectorizer': [vectorizer.__class__.__name__],
        'Ngram': [ngram],
        'TopKFeatures': [topk],
        'TrainAccuracy': [train_acc],
        'TestAccuracy': [test_acc]
    }), ignore_index=True)

In [ ]:
TOP_KS = np.arange(9000, 12001, 1500)
TOP_KS

array([ 9000, 10500, 12000])

# Classifying on full text

## CountVectorizer on Unigrams

In [ ]:
for i in (TOP_KS):
    print(f"For top {i} features selected:\n")
    write_df(processed_split_full_text, 1, 'full_text', SGDClassifier, CountVectorizer, topk=i)
    print('\n')

For top 9000 features selected:

Train classification accuracy = 0.622963368641167,
 Test classification accuracy = 0.5572427870852107


For top 10500 features selected:

Train classification accuracy = 0.625434087203296,
 Test classification accuracy = 0.5608025690174504


For top 12000 features selected:

Train classification accuracy = 0.6349871174120312,
 Test classification accuracy = 0.5615244828358766




## TfidfVectorizer on Unirams and Bigrams

In [ ]:
for i in (TOP_KS):
    print(f"For top {i} features selected:\n")
    write_df(processed_split_full_text, 1, 'full_text', SGDClassifier, TfidfVectorizer, topk=i)
    print('\n')

For top 9000 features selected:

Train classification accuracy = 0.5881990515427988,
 Test classification accuracy = 0.5354111174728038


For top 10500 features selected:

Train classification accuracy = 0.5885226721101305,
 Test classification accuracy = 0.5363321799307958


For top 12000 features selected:

Train classification accuracy = 0.5965322811515913,
 Test classification accuracy = 0.5399168554429813




# Showing top 5 results by test accuracy and saving the table to csv file

In [ ]:
results_df.sort_values(by='TestAccuracy', ascending=False).head(5)

,Classifier,By,Preprocessed,Vectorizer,Ngram,TopKFeatures,TrainAccuracy,TestAccuracy
2,SGDClassifier,full_text,1,CountVectorizer,"(1, 1)",12000,0.634987,0.561524
1,SGDClassifier,full_text,1,CountVectorizer,"(1, 1)",10500,0.625434,0.560803
0,SGDClassifier,full_text,1,CountVectorizer,"(1, 1)",9000,0.622963,0.557243
5,SGDClassifier,full_text,1,TfidfVectorizer,"(1, 1)",12000,0.596532,0.539917
4,SGDClassifier,full_text,1,TfidfVectorizer,"(1, 1)",10500,0.588523,0.536332


In [ ]:
results_df.to_csv('/content/drive/MyDrive/SDLC/news_analysis_project/resultsSGD_for_all.csv')